# Tensor flow tutorial

### This is my first time trying out TF, just a bunch of random operation and stuff getting used to it 

In [1]:
%config IPCompleter.greedy=True
    
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

In [2]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
print(dataset)
for element in dataset:
    print(element.numpy())

<TensorSliceDataset shapes: (), types: tf.int32>
8
3
0
8
2
1


In [9]:
print(dataset.reduce(0, lambda state, value: state + value).numpy())

22


In [10]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))
dataset1.element_spec
for elem in dataset1:
    print(elem)

tf.Tensor([0.044  0.3233 0.2338 0.9092 0.0832 0.1227 0.1113 0.1348 0.7699 0.0665], shape=(10,), dtype=float32)
tf.Tensor([0.0184 0.5239 0.1563 0.2238 0.8273 0.5137 0.864  0.236  0.2664 0.2294], shape=(10,), dtype=float32)
tf.Tensor([0.0868 0.0135 0.1834 0.0036 0.7822 0.62   0.1627 0.6538 0.9459 0.254 ], shape=(10,), dtype=float32)
tf.Tensor([0.6285 0.2551 0.1855 0.7135 0.3459 0.8134 0.3907 0.5594 0.2759 0.6435], shape=(10,), dtype=float32)


In [17]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))
for elem in dataset2:
    print(elem)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.6818397>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([12,  4, 18, 31, 96, 31, 74, 47, 12, 63, 97, 26,  5, 81, 56, 45, 76,
       22,  2, 35, 38, 48, 97,  6, 18,  7,  9, 12, 11, 47, 89, 76, 10, 81,
       68, 84,  6, 87, 74, 54, 91, 10, 62,  7, 81, 30, 57, 77, 25, 57,  0,
       87, 17, 25, 14, 30, 85, 33, 78, 77, 30, 36, 54, 76, 91, 82, 61, 13,
        5, 54,  6, 90, 30, 44, 59, 69,  8, 50,  9, 75, 35, 14, 26, 52, 93,
       99,  9, 90, 84, 48, 87,  9, 67, 20, 97, 31, 35, 71, 57, 77])>)
(<tf.Tensor: shape=(), dtype=float32, numpy=0.33408022>, <tf.Tensor: shape=(100,), dtype=int32, numpy=
array([26, 44, 36, 56, 12, 62, 64, 96,  7, 94, 42, 93, 97, 34, 43, 83, 40,
       58, 34, 20, 48, 33, 21, 54, 90, 90, 65, 75, 88, 79, 17, 71, 34,  8,
       13, 70, 58, 52, 28, 95, 64, 75, 39, 94, 95, 67,  5, 72, 99,  1, 61,
       39, 49, 88, 32, 33, 23, 15, 72, 98, 18, 88, 53, 95, 38, 63, 14, 96,
        5, 39, 55,  7, 44, 90, 66, 95, 25, 37, 75,

In [18]:
train, test = tf.keras.datasets.fashion_mnist.load_data()


4423680/4422102 [==============================] - 2s 0us/step


In [21]:
images, labels = train
images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset



<TensorSliceDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [33]:
range_ds = tf.data.Dataset.range(100000)
batches = range_ds.batch(10, drop_remainder=True)

for batch in batches.take(5):
  print(batch.numpy())

@tf.autograph.experimental.do_not_convert
def dense_1_step(batch):
  # Shift features and labels one step relative to each other.
  return batch[:-1], batch[1:]

predict_dense_1_step = batches.map(dense_1_step)

for features, label in predict_dense_1_step.take(3):
  print(features.numpy(), " => ", label.numpy())

batches = range_ds.batch(15, drop_remainder=True)
@tf.autograph.experimental.do_not_convert
def label_next_5_steps(batch):
  return (batch[:-5],   # Take the first 5 steps
          batch[-5:])   # take the remainder

predict_5_steps = batches.map(label_next_5_steps)

for features, label in predict_5_steps.take(3):
  print(features.numpy(), " => ", label.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]
[40 41 42 43 44 45 46 47 48 49]
[0 1 2 3 4 5 6 7 8]  =>  [1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18]  =>  [11 12 13 14 15 16 17 18 19]
[20 21 22 23 24 25 26 27 28]  =>  [21 22 23 24 25 26 27 28 29]
[0 1 2 3 4 5 6 7 8 9]  =>  [10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]  =>  [25 26 27 28 29]
[30 31 32 33 34 35 36 37 38 39]  =>  [40 41 42 43 44]


In [35]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/download.tensorflow.org/data/creditcard.zip',
    fname='creditcard.zip',
    extract=True)

csv_path = zip_path.replace('.zip', '.csv')



In [36]:
creditcard_ds = tf.data.experimental.make_csv_dataset(
    csv_path, batch_size=1024, label_name="Class",
    # Set the column types: 30 floats and an int.
    column_defaults=[float()]*30+[int()])

In [39]:
@tf.autograph.experimental.do_not_convert
def count(counts, batch):
  features, labels = batch
  class_1 = labels == 1
  class_1 = tf.cast(class_1, tf.int32)

  class_0 = labels == 0
  class_0 = tf.cast(class_0, tf.int32)

  counts['class_0'] += tf.reduce_sum(class_0)
  counts['class_1'] += tf.reduce_sum(class_1)

  return counts

counts = creditcard_ds.take(10).reduce(
    initial_state={'class_0': 0, 'class_1': 0},
    reduce_func = count)

counts = np.array([counts['class_0'].numpy(),
                   counts['class_1'].numpy()]).astype(np.float32)

fractions = counts/counts.sum()
print(fractions)

[0.9958 0.0042]
